In [4]:

import iesta.machine_learning.dataloader
#from .autonotebook import tqdm as notebook_tqdm
import iesta.stats.significance
import matplotlib.pyplot as plt

%matplotlib inline
import seaborn as sns

sns.reset_defaults()
sns.set(
    rc={'figure.figsize':(3,2)}, 
    #style="white" # nicer layout
)


In [2]:
feature_dfs = iesta.machine_learning.dataloader.load_features_df()
significance = iesta.stats.significance.run_all_significance_test(feature_dfs)

/home/elba_ro/repos/conf22-style-transfer/iesta/../data/splitted_conservative_debate_arguments_effect_test0.3_random2.parquet


iesta        INFO     File already created. Loading file...


/home/elba_ro/repos/conf22-style-transfer/iesta/../data/splitted_liberal_debate_arguments_effect_test0.3_random2.parquet


iesta        INFO     File already created. Loading file...


316: ../data/extracted_features//conservative_style-features_1000/conservative_batch1000_72_style-features.parquet
16: ../data/extracted_features//conservative_transformer-features_100/conservative_batch100_714_transformer-features.parquet
792: ../data/extracted_features//liberal_style-features_1000/liberal_batch1000_47_style-features.parquet
92: ../data/extracted_features//liberal_transformer-features_100/liberal_batch100_468_transformer-features.parquet


In [ ]:
significance.keys()

In [ ]:
significance['conservative_binary_effect_transformers'][0]

In [ ]:
significance['conservative_effect_undersampled_empath-mpqa'][0]

In [ ]:
type(significance["conservative_effect__transformers"][0]['effective okay'].unique().tolist())

In [ ]:
key= "liberal_effect_empath-mpqa"
cons_effect_tramsformers = significance[key][0].reset_index()
cons_effect_tramsformers = cons_effect_tramsformers[(~cons_effect_tramsformers["feature"].str.endswith("_count")) & (~cons_effect_tramsformers["feature"].str.endswith("_ratio"))]
cons_effect_tramsformers

In [ ]:
liberal_effect_tramsformers = significance['liberal_effect__transformers'][0].reset_index()
liberal_effect_tramsformers = liberal_effect_tramsformers[(~liberal_effect_tramsformers["feature"].str.endswith("_count")) & (~liberal_effect_tramsformers["feature"].str.endswith("_ratio"))]
liberal_effect_tramsformers